In [1]:
import torch 
import numpy as np
import pandas as pd 
import utilities
from math import ceil
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout


In [2]:
data = pd.read_csv("Datasets/sp500_sentiment_non_weighted.csv")
data

,Unnamed: 0,date,Volume,Close,Positive,Negative,Neutral,Price_change
0,0,2010-01-05,1.141819e+07,1136.520020,0.275458,0.357004,0.367538,1
1,1,2010-01-06,1.078500e+07,1137.140015,0.292846,0.177264,0.529890,1
2,2,2010-01-07,1.202273e+07,1141.689941,0.418571,0.011353,0.570076,1
3,3,2010-01-08,1.621044e+07,1144.979980,0.075816,0.252077,0.672107,1
4,4,2010-01-11,1.167700e+07,1146.979980,0.602344,0.010513,0.387143,1
...,...,...,...,...,...,...,...,...
2612,2612,2020-06-05,3.789298e+07,3193.929932,0.507138,0.087936,0.404926,1
2613,2613,2020-06-08,5.888138e+07,3232.389893,0.510386,0.121527,0.368087,1
2614,2614,2020-06-09,6.799510e+07,3207.179932,0.322274,0.249364,0.428363,-1
2615,2615,2020-06-10,8.681144e+07,3190.139893,0.418343,0.205621,0.376035,-1


In [3]:
train = pd.DataFrame(data[0:int(len(data)*0.70)])
test = pd.DataFrame(data[int(len(data)*0.70): int(len(data))])

print(train.shape)
print(test.shape)

(1831, 8)
(786, 8)


In [4]:
train.head()

,Unnamed: 0,date,Volume,Close,Positive,Negative,Neutral,Price_change
0,0,2010-01-05,1.141819e+07,1136.520020,0.275458,0.357004,0.367538,1
1,1,2010-01-06,1.078500e+07,1137.140015,0.292846,0.177264,0.529890,1
2,2,2010-01-07,1.202273e+07,1141.689941,0.418571,0.011353,0.570076,1
3,3,2010-01-08,1.621044e+07,1144.979980,0.075816,0.252077,0.672107,1
4,4,2010-01-11,1.167700e+07,1146.979980,0.602344,0.010513,0.387143,1


In [5]:
test.head()

,Unnamed: 0,date,Volume,Close,Positive,Negative,Neutral,Price_change
1831,1831,2017-04-28,1.628920e+07,2384.199951,0.056357,0.155713,0.787930,-1
1832,1832,2017-05-01,3.851360e+07,2388.330078,0.089706,0.232008,0.678287,1
1833,1833,2017-05-02,6.844709e+07,2391.169922,0.140315,0.138546,0.721139,1
1834,1834,2017-05-03,5.214252e+07,2388.129883,0.159640,0.062907,0.777454,-1
1835,1835,2017-05-04,1.954496e+07,2389.520020,0.363036,0.178503,0.458461,1


In [6]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))

In [7]:
train_close = train.iloc[:, 2:7].values
test_close = test.iloc[:, 2:7].values

In [8]:
data_training_array = scaler.fit_transform(train_close)
data_training_array
data_testing_array=scaler.fit_transform(test_close)
data_testing_array

array([[0.03409419, 0.12779112, 0.041999  , 0.19629166, 0.92830984],
       [0.09846646, 0.13138644, 0.0990134 , 0.30384424, 0.79198705],
       [0.18516783, 0.13385856, 0.18553835, 0.17209041, 0.84526676],
       ...,
       [0.18385868, 0.8442046 , 0.49662323, 0.32831104, 0.48124835],
       [0.2383596 , 0.82937105, 0.66086871, 0.26664741, 0.41618796],
       [0.04106318, 0.66568026, 0.09221664, 0.93190923, 0.24298817]])

In [9]:
x_train = []
y_train = [] 
x_test = []
y_test = []

for i in range(100, data_training_array.shape[0]):
    x_train.append(data_training_array[i-100: i])
    y_train.append(data_training_array[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train) 


for i in range(100, data_training_array.shape[0]):
    x_test.append(data_training_array[i-100: i])
    y_test.append(data_training_array[i, 0])

x_test, y_test = np.array(x_test), np.array(y_test) 

In [10]:
x_train.shape

(1731, 100, 5)

In [11]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 5))  # 5 features
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 5))      # 5 features

LSTM Model

In [12]:
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.models import Sequential

In [13]:
model = Sequential()
model.add(LSTM(units = 50, activation = 'relu', return_sequences=True
              ,input_shape = (x_train.shape[1], 1)))
model.add(Dropout(0.2))


model.add(LSTM(units = 60, activation = 'relu', return_sequences=True))
model.add(Dropout(0.3))


model.add(LSTM(units = 80, activation = 'relu', return_sequences=True))
model.add(Dropout(0.4))


model.add(LSTM(units = 120, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(units = 1))

c:\Users\Asus\anaconda3\envs\venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [14]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 100, 50)        │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100, 50)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 100, 60)        │        26,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 100, 60)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 100, 80)        │        45,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 100, 80)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 120)            │        96,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 120)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           121 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 178,761 (698.29 KB)

 Trainable params: 178,761 (698.29 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
# Compiling the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['MAE'])

# Training the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=50)

# Evaluate model on test data
model.evaluate(x_test, y_test)

Epoch 1/50


ValueError: Exception encountered when calling LSTMCell.call().

[1mDimensions must be equal, but are 5 and 1 for '{{node sequential_1/lstm_1/lstm_cell_1/MatMul}} = MatMul[T=DT_FLOAT, grad_a=false, grad_b=false, transpose_a=false, transpose_b=false](sequential_1/lstm_1/strided_slice_2, sequential_1/lstm_1/lstm_cell_1/Cast/ReadVariableOp)' with input shapes: [?,5], [1,200].[0m

Arguments received by LSTMCell.call():
  • inputs=tf.Tensor(shape=(None, 5), dtype=float32)
  • states=('tf.Tensor(shape=(None, 50), dtype=float32)', 'tf.Tensor(shape=(None, 50), dtype=float32)')
  • training=True

In [ ]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], x_train.shape[2]))
x_train
